# California Housing Price 
reference : exercise in chapter 2 of 'Hands-On Machine Learning with Scikit-learn and Tensorflow' by Aurélien Géron. 

##### Tip> shortcuts for Jupyter Notebook
* Ctrl + Enter : run cell
* Shift + Enter : run cell and select below

## 1. Data Load

Load the data by using *read_csv()* method in __Pandas__ module. Then, let's take a look at the top 10 rows using the *head()* method. 

In [ ]:
# Data load
import pandas as pd

housing = pd.read_csv('housing.csv')
housing.head(10)

Let's see the distribution of the data by using __matplotlib__ module briefly.

In [ ]:
# figures plotting with data
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

housing.plot(kind="scatter", x="longitude", y="latitude", alpha=0.4,
    s=housing["population"]/50, label="population", figsize=(10,7),
    c="median_house_value", cmap=plt.get_cmap("jet"), colorbar=True,
    sharex=False)

plt.legend()

To better understand the characteristics of each feature, let's apply the *info()* method.

In [ ]:
# check a structure of the data
housing.info()

Let’s look at how much each attribute correlates with the *median house value*:

In [ ]:
# correlation between the median_house_value and other features
corr_matrix = housing.corr()
corr_matrix['median_house_value'].sort_values(ascending=False)

## 2. Prepare the Data

this step consists of 'pre-processing', 'train-test seperation', and 'feature-label seperation'.

### 2-1) Pre-processing 

#### 2-1.1) Data cleaning
Most Machine Learning algorithms cannot work with missing features, so let’s replace the empty values of 'total_bedrooms' with the median value.

In [ ]:
# replace the empty values with the median
median =housing["total_bedrooms"].median()
housing["total_bedrooms"] = housing["total_bedrooms"].fillna(median) 
housing.info()

#### 2-1.2) Attributes combinations
*rooms_per_household* is more meaningful than *total_rooms*. Also, *bedrooms_per_room* is more meaningful than *total_bedrooms*.

In [ ]:
# Attributes combinations
housing["rooms_per_household"] = housing["total_rooms"]/housing["households"]
housing["bedrooms_per_room"] = housing["total_bedrooms"]/housing["total_rooms"]
del housing["total_rooms"], housing["total_bedrooms"]

housing.info()

#### 2-1.3) Feature Scaling
Machine Learning algorithms don’t perform well when the input numerical attributes have very different scales.

__Scikit-Learn__ provides a transformer called *StandardScaler* for *standardization*.

In [ ]:
# feature standardization
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

# delete columns of text type and target variable
col_list = list(housing) 
col_list.remove('ocean_proximity') # text type
col_list.remove('median_house_value') # target variable needs not to be scaled

# generate a new dataframe that consist of numeric type only
housing_numeric = housing[col_list]
housing_scaled = scaler.fit_transform(housing_numeric)
# Data type conversion from 'Series' to 'DataFrame'
housing_scaled_df = pd.DataFrame(housing_scaled, index=housing_numeric.index, columns=housing_numeric.columns)

# Concatenate 
housing = pd.concat([housing_scaled_df, housing['median_house_value'], housing['ocean_proximity']], axis=1)
housing.head()

#### 2-1.4) Handling Text and Categorical Attributes
Most Machine Learning algorithms prefer to work with numbers anyway, so let’s convert the 'ocean_proximity' to numbers.

__Pandas__ provides a *get_dummies* method to convert integer categorical values into one-hot vectors. 

In [ ]:
# One-hot encoding
housing = pd.get_dummies(housing)
housing.head(10)

### 2-2) Training and Test Set Seperation
__Scikit-Learn__ provides *train_test_split* function to split dataset into multiple subsets in various ways. 

In [ ]:
# training - test seperation
from sklearn.model_selection import train_test_split

train_set, test_set = train_test_split(housing, test_size=0.2, random_state=42)

print('# of train_set : %.0f, # of test_set : %.0f' %(train_set.shape[0], test_set.shape[0]))

### 2-3) Features and Target Value Seperation of the Training Set
It’s time to prepare the data for your Machine Learning algorithms. 

Let’s separate the features and target value to generate the model H(X).

In [ ]:
# feature and label seperation of training set
train_set_features = train_set.drop('median_house_value',axis=1)
train_set_target = train_set["median_house_value"].copy()

## 3. Linear Regression
generate the linear regression model by using *LinearRegression* function from __Scikit-learn__.

For calculating our RMSE, *mean_square_error* function will be used from __scikit-learn__. Also, __numpy__ module will be used to use sqaure-root operation.

 $$RMSE = \sqrt{\sum{(y - \widehat y)^2}\over N}$$
 <br/>
 
$y$ : actual median_house_value, $\widehat y$ : median_house_value predicted. $N$ : total number of data<br/>

In [ ]:
def RMSE_calculation(param1, param2):
    mse = mean_squared_error(param1, param2)
    rmse = np.sqrt(mse)

    return rmse

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np # for a sqaure root calcuation

# generate model by using training set
lin_reg = LinearRegression()
lin_reg.fit(train_set_features, train_set_target) 

# Feature and target value Seperation of the test set
test_set_features = test_set.drop('median_house_value',axis=1)
test_set_target = test_set["median_house_value"].copy()

# target value predicted from our model
train_final_predictions = lin_reg.predict(train_set_features)
test_final_predictions = lin_reg.predict(test_set_features)

# RMSE
train_final_rmse = RMSE_calculation(train_set_target, train_final_predictions)
test_final_rmse = RMSE_calculation(test_set_target, test_final_predictions)

print('train_RMSE : %.2f , test_RMSE : %.2f' %(train_final_rmse, test_final_rmse))

## 4. Linear Regression with Gradient Descent

To compute matrix calculations easily, convert data types from dataframe to numpy array. 

In [ ]:
train_set_features_array = train_set_features.values
train_set_target_array =np.expand_dims(train_set_target,axis=1)

test_set_features_array = test_set_features.values
test_set_target_array = np.expand_dims(test_set_target,axis=1)

train_y = train_set_target.values.reshape(-1,1)
test_y = test_set_target.values.reshape(-1,1)

Generate the augmented matrix $X_{b}$ by adding bias column which consists of all 1s.

In [ ]:
train_bias_column = np.ones([train_set_features_array.shape[0],1])
train_X_b = np.concatenate((train_set_features_array, train_bias_column),axis=1)

test_bias_column = np.ones([test_set_features_array.shape[0],1])
test_X_b = np.concatenate((test_set_features_array, test_bias_column),axis=1)

### 4-1) Batch Gradient Descent

The gradient descent mothod is as follows.

$$ \theta \leftarrow \theta - \eta\nabla_{\theta} J(\theta) \quad ,\,where \; \eta = learning\, rate$$ 


Recall that 

$$ \nabla_{\theta} J(\theta) = X^{T}X\theta - X^{T}y $$
$$ = X^{T}(X\theta-y)$$

It is batch gradient descent that $X$ contains all training instances.

At first, initialize $\theta$ and store the training RMSE and validation RMSE for this initialized $\theta$.

In [ ]:
# initialize theta 
feature_num = train_X_b.shape[1]
theta = np.random.randn(feature_num,1)

GD_train_rmse = []
GD_val_rmse = []

initialized_theta_train_RMSE = RMSE_calculation(np.dot(train_X_b,theta), train_y)
initialized_theta_val_RMSE = RMSE_calculation(np.dot(test_X_b,theta), test_y)

GD_train_rmse.append(initialized_theta_train_RMSE)
GD_val_rmse.append(initialized_theta_val_RMSE)

print('RMSE of the initialized theta - train_RMSE : %.2f, val_RMSE : %.2f' %(initialized_theta_train_RMSE, initialized_theta_val_RMSE))

Execute your batch gradient descent.

In [ ]:
train_num = train_X_b.shape[0]

batch_size = train_num
n_epoch = 10000
eta = 0.001 # learning rate

for epoch in range(n_epoch):
    gradient = 2.0 / batch_size * np.dot(train_X_b.T , np.dot(train_X_b, theta) - train_y)
    theta = theta - eta * gradient
    
    # train error
    GD_train_predictions = np.dot(train_X_b, theta)
    GD_train_final_rmse = RMSE_calculation(train_y, GD_train_predictions)
    GD_train_rmse.append(GD_train_final_rmse)
    
    # val error
    GD_val_predictions = np.dot(test_X_b, theta)
    GD_val_final_rmse = RMSE_calculation(test_y, GD_val_predictions)
    GD_val_rmse.append(GD_val_final_rmse)
    
    if epoch%100 == 0:
        print('%d번째 train_RMSE : %.2f, val_RMSE : %.2f' %(epoch, GD_train_final_rmse, GD_val_final_rmse))

Check for changes of RMSE as epoch increases.

In [ ]:
plt.plot(GD_train_rmse, "r+", linewidth=2, label="train")
plt.plot(GD_val_rmse, "b-", linewidth=3, label="val")

plt.ylim([60000, 250000])
plt.legend(loc='upper right')
plt.xlabel('epoch')
plt.ylabel('RMSE')

### 4-2) Mini-Batch Gradient Descent

It is mini-batch gradient descent that  𝑋  contains only a small set of randomly selected training instances.<br/>
It is important to shuffle the order of train instances after each epochs.

Initialize  𝜃  and store the training RMSE and validation RMSE for this initialized  𝜃 .

In [ ]:
theta = np.random.randn(feature_num,1)
theta_bkp = theta # back-up the theta values

GD_train_rmse = []
GD_val_rmse = []

GD_batch_train_rmse = []
GD_batch_val_rmse = []

initialized_theta_train_RMSE = RMSE_calculation(np.dot(train_X_b,theta), train_y)
initialized_theta_val_RMSE = RMSE_calculation(np.dot(test_X_b,theta), test_y)

GD_train_rmse.append(initialized_theta_train_RMSE)
GD_val_rmse.append(initialized_theta_val_RMSE)
GD_batch_train_rmse.append(initialized_theta_train_RMSE)
GD_batch_val_rmse.append(initialized_theta_val_RMSE)

print('RMSE of the initialized theta - train_RMSE : %.2f, val_RMSE : %.2f' %(initialized_theta_train_RMSE, initialized_theta_val_RMSE))

Execute your mini-batch gradient descent.

In [ ]:
batch_size = 128
n_epoch = 100
eta = 0.001 # learning rate

for epoch in range(n_epoch):
    
    # shuffle
    shuffle_indices = np.random.permutation(train_num)
    feature_shuffled = train_X_b[shuffle_indices,:]
    target_shuffled = train_y[shuffle_indices,:]
    
    for i in range(0, train_num, batch_size):
        batch_x = feature_shuffled[i:i+batch_size,:]
        batch_y = target_shuffled[i:i+batch_size,:]

        gradient = 2.0/batch_size * np.dot(batch_x.T , np.dot(batch_x, theta) - batch_y)
        theta = theta - eta * gradient
        
        # train error after batch update 
        train_batch_predictions = np.dot(train_X_b, theta)
        train_batch_rmse = RMSE_calculation(train_y, train_batch_predictions)
        GD_batch_train_rmse.append(train_batch_rmse)
        
        # test error after batch update
        val_batch_predictions = np.dot(test_X_b, theta)
        val_batch_rmse = RMSE_calculation(test_y, val_batch_predictions)
        GD_batch_val_rmse.append(val_batch_rmse)
        
    # train error after epoch 
    GD_train_predictions = np.dot(train_X_b, theta)
    GD_train_final_rmse = RMSE_calculation(train_y, GD_train_predictions)
    GD_train_rmse.append(GD_train_final_rmse)
    
    # val error after epoch 
    GD_val_predictions = np.dot(test_X_b, theta)
    GD_val_final_rmse = RMSE_calculation(test_y, GD_val_predictions)
    GD_val_rmse.append(GD_val_final_rmse)
    
    if epoch%10 == 0:
        print('%d번째 train_RMSE : %.2f, val_RMSE : %.2f' %(epoch, GD_train_final_rmse, GD_val_final_rmse))

Check for changes of RMSE as epoch increases.

In [ ]:
plt.plot(GD_train_rmse, "r+", linewidth=2, label="train")
plt.plot(GD_val_rmse, "b-", linewidth=3, label="val")

plt.ylim([60000, 250000])
plt.legend(loc='upper right')
plt.xlabel('epoch')
plt.ylabel('RMSE')

Check for changes of RMSE as the number of updates increases.

In [ ]:
plt.plot(GD_batch_train_rmse, "r+", linewidth=2, label="train")
plt.plot(GD_batch_val_rmse, "b-", linewidth=3, label="val")

plt.ylim([60000, 250000])
plt.xlim([0,2000])
plt.legend(loc='upper right')
plt.xlabel('updates')
plt.ylabel('RMSE')

### 4-3) Stochastic Gradient Descent

It is stochastic gradient descent that $𝑋$ contains only one training instance selected randomly.

Initialize 𝜃 and store the training RMSE and validation RMSE for this initialized 𝜃 

In [ ]:
theta = np.random.randn(feature_num,1)
theta_bkp = theta # back-up the theta values

GD_train_rmse = []
GD_val_rmse = []

GD_batch_train_rmse = []
GD_batch_val_rmse = []

initialized_theta_train_RMSE = RMSE_calculation(np.dot(train_X_b,theta), train_y)
initialized_theta_val_RMSE = RMSE_calculation(np.dot(test_X_b,theta), test_y)

GD_train_rmse.append(initialized_theta_train_RMSE)
GD_val_rmse.append(initialized_theta_val_RMSE)
GD_batch_train_rmse.append(initialized_theta_train_RMSE)
GD_batch_val_rmse.append(initialized_theta_val_RMSE)

print('RMSE of the initialized theta - train_RMSE : %.2f, val_RMSE : %.2f' %(initialized_theta_train_RMSE, initialized_theta_val_RMSE))

Execute your stochastic gradient descent algorithm.<br/>

In [ ]:
######################## Q1. ##################################
#  TO DO : Fill in the blank with your stochastic gradient descent code.
# set the hyper-parameters as follows.
# parameters setting : batch size = 1 , epoch = 50 , learning rate = 0.001

















    
    ######################## Q2. ##################################
    
    
    ##############################################################
    
    
    
##############################################################

In [ ]:
######################## Q3. ##################################
# TO DO : plot the learning curve of each epoch.





##############################################################

In [ ]:
######################## Q4. ##################################
# TO DO : plot the learning curve of each updates.





##############################################################

### 4-4) Stochastic Gradient Descent using SGDRegressor

Scikit-Learn provides a function for stochastic gradient descent, SGDRegressor().<br/>
SGDRegressor() has 2 methods, 'fit()' and 'partial_fit()'. <br/>
If you use 'fit()' method, you can check the RMSE after total epoches. In the case of 'partial_fit()' method, you can check the RMSE of each epoch.<br/>
To plot the changes of RMSE depending on each epoch, please use 'partial_fit()' method.<br/>

reference : https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDRegressor.html

In [ ]:
SGDfunc_train_rmse = []
SGDfunc_val_rmse = []

SGDfunc_initialized_theta_train_RMSE = RMSE_calculation(np.dot(train_X_b,theta_bkp), train_y)
SGDfunc_initialized_theta_val_RMSE = RMSE_calculation(np.dot(test_X_b,theta_bkp), test_y)

SGDfunc_train_rmse.append(SGDfunc_initialized_theta_train_RMSE)
SGDfunc_val_rmse.append(SGDfunc_initialized_theta_val_RMSE)


print('RMSE of the initialized theta - train_RMSE : %.2f, val_RMSE : %.2f' 
      %(SGDfunc_initialized_theta_train_RMSE, SGDfunc_initialized_theta_val_RMSE))

Fill in the blanks with your code.

In [ ]:
######################## Q1. ##################################
#  TO DO : import SGDRegressor() function.
from sklearn.linear_model 

##############################################################



######################## Q2. ##################################
# TO DO : set the number of n_epoch to 50
n_epoch = 
##############################################################



######################## Q3. ##################################
# TO DO : set the parameters of SGDRegressor() as follows.
# parameters setting : penalty='none', learning_rate='constant', eta0=0.001

SGD_model = 
##############################################################


for epoch in range(n_epoch):   
    ######################## Q4. ##################################
    # TO DO : execute 'partial_fit()' method.
    
    ##############################################################
    
    # train error
    SGD_train_predictions = SGD_model.predict(train_set_features)
    SGD_train_final_rmse = RMSE_calculation(SGD_train_predictions, train_set_target)
    SGDfunc_train_rmse.append(SGD_train_final_rmse)
    
    # val error
    SGD_val_predictions = SGD_model.predict(test_set_features)
    SGD_val_final_rmse = RMSE_calculation(SGD_val_predictions, test_set_target)
    SGDfunc_val_rmse.append(SGD_val_final_rmse)
    
    if epoch%5 == 0:
           print('%d번째 train_RMSE : %.2f, val_RMSE : %.2f' %(epoch, SGD_train_final_rmse, SGD_val_final_rmse))

    # command to hide the warning box
    import warnings
    warnings.filterwarnings(action = 'ignore')


In [ ]:
######################## Q5. ##################################
# TO DO : plot the learning curve as epoch changes.




##############################################################